In [1]:
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
data = []
for page in range(1, 73):
    url = 'https://www.bezrealitky.cz/vyhledat?offerType=PRONAJEM&estateType=BYT&regionOsmIds=R435541&osm_value=Praha%2C+%C4%8Cesko&currency=CZK&page='
    webpage = requests.get(url + str(page))
    soup = BeautifulSoup(webpage.content, 'html.parser')
    
    #-----------------removes not necessary "Výhodná nabídka" box-----------------
    section_tags = soup.find_all('section', class_="box Section_section__gjwvr section PropertyCardCarousel_propertyCardCarouselSection__ts7NT mb-0 bg-grey-ultra-light Box_box--rounded-lg__uS5ZG")
    for section in section_tags:
        section.decompose()    
        
    articles = soup.find_all('article', class_="PropertyCard_propertyCard__moO_5 propertyCard PropertyCard_propertyCard--landscape__XvPmC")
    for article in articles:
        
        street_district_tag = article.find('span', class_="PropertyCard_propertyCardAddress__hNqyR text-subheadline text-truncate")        
        size_tag = article.find('ul', class_="FeaturesList_featuresList__75Wet featuresList mt-3")
        rent_price_tag = article.find('span', class_="PropertyPrice_propertyPriceAmount__WdEE1")       
        add_fee_tag = article.find('span', class_="PropertyPrice_propertyPriceAdditional__5jYQ6")       
        
        #-----------------extracts text from tags-----------------
        street_district = street_district_tag.get_text() if street_district_tag else None
        rent_price = rent_price_tag.get_text() if rent_price_tag else None
        add_fee = add_fee_tag.get_text() if add_fee_tag else None 
        
        if size_tag and len(size_tag.contents) > 1:
            room = size_tag.contents[0].get_text() if size_tag.contents[0] else None
            area = size_tag.contents[1].get_text() if size_tag.contents[1] else None

        #-----------------separates street and district-----------------
        street = street_district.split(',')[0] 
        district = street_district.split(',')[-1].split('- ')[-1]

        data.append([street, district, rent_price, add_fee, room, area])




In [3]:
#-----------------saves data in csv file-----------------
file_name = "bezrealitka_data.csv"

# Write the list data to a CSV file
with open(file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write a header row
    writer.writerow(['Street', 'District', 'Price', 'Fee', 'Room', 'Area'])
    # Write the data rows
    writer.writerows(data)



In [6]:
len(data)

1071